# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

# Описание данных

**Признаки**
* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

**Целевой признак**
* Exited — факт ухода клиента

## Подготовка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle

Загрузим датафрейм и выведем его первые 5 строк

In [2]:
try:   
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')
    
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Узнаем общую информацию про полученный датафрейм

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data['Tenure'].isnull().sum()

909

В столбце Tenure оказалось 909 пропущенных значений. У этой проблемы есть 2 решения: Заполнить пропуски своими значениями(допустим, предположить что все клиенты с “пропусками” это новое клиенты и заполнить все нулями) или просто удалить все пропуски. По моему мнению искусственное заполнение данных может повлиять на итоговый результат обучения, поэтому удалим все пропуски из нашего датафрейма, так как количество удаленных данных будет меньше 10% от общего количества всех данных, что можно считать позволительным.  

In [5]:
old_data = data.shape[0]
data = data.dropna()
procent_del = (1 - data.shape[0] / old_data) * 100
print(f'Удалено {procent_del:.2f}% данных')

Удалено 9.09% данных


Узнаем общую информацию про полученный датафрейм после преобразования

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        9091 non-null   int64  
 1   CustomerId       9091 non-null   int64  
 2   Surname          9091 non-null   object 
 3   CreditScore      9091 non-null   int64  
 4   Geography        9091 non-null   object 
 5   Gender           9091 non-null   object 
 6   Age              9091 non-null   int64  
 7   Tenure           9091 non-null   float64
 8   Balance          9091 non-null   float64
 9   NumOfProducts    9091 non-null   int64  
 10  HasCrCard        9091 non-null   int64  
 11  IsActiveMember   9091 non-null   int64  
 12  EstimatedSalary  9091 non-null   float64
 13  Exited           9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


Для обучения модели информация из колонок RowNumber, CustomerId, Surname не несет никакой пользы, поэтому удалим их из нашего датафрейма 

In [7]:
del_columns = ['RowNumber', 'CustomerId','Surname']
data = data.drop(del_columns, axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Разделим исходные данные для обучения

In [8]:
features = data.drop('Exited', axis=1)
target = data['Exited']

Преобразуем категориальные признаки методом OHE

In [9]:
features = pd.get_dummies(features, drop_first=True)
features.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,1,0


Разделим исходные данные на обучающую, валидационную и тестовую выборки

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

Исходные данные разделены на 60%, 20% и 20%

In [11]:
print('Обучающуя выборка:', features_train.shape, target_train.shape)
print('Валидационнуя выборка:', features_valid.shape, target_valid.shape)
print('Тестовая выборка:', features_test.shape, target_test.shape)

Обучающуя выборка: (5454, 11) (5454,)
Валидационнуя выборка: (1819, 11) (1819,)
Тестовая выборка: (1818, 11) (1818,)


Стандартизируем количественные признаки

In [12]:
pd.options.mode.chained_assignment = None

standart_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[standart_columns])
features_train[standart_columns] = scaler.transform(features_train[standart_columns])
features_valid[standart_columns] = scaler.transform(features_valid[standart_columns])
features_test[standart_columns] = scaler.transform(features_test[standart_columns])

Подготовка данных на этом закончена, можно приступать к поиску наилучшей модели   

## Исследование задачи

Исследуем баланс классов    

In [13]:
print(f'Количество значений равное 0 - {target[target == 0].count()}, \
Количество значений равное 1 - {target[target == 1].count()}')

Количество значений равное 0 - 7237, Количество значений равное 1 - 1854


Наблюдается дисбаланс классов, найдем наилучшую модель без учета дисбаланса

Обучим и найдем F1-меру для логистической регрессии 

In [14]:
%%time

best_f1_regression = 0
for c_param in np.arange(0.5, 1.5, 0.05):
    for iter in range(100, 3001, 100):
        model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter, C=c_param) 
        model_regression.fit(features_train, target_train)
        predictions_valid = model_regression.predict(features_valid)
        f1_regression = f1_score(target_valid, predictions_valid)
        if f1_regression > best_f1_regression:
            best_model_regression = model_regression
            best_f1_regression = f1_regression
            best_max_iter = iter
            best_c_param = c_param
print('Параметр F1:', best_f1_regression, 'Параметр max_iter:', best_max_iter, 'Параметр C:', best_c_param)

Параметр F1: 0.32719836400818 Параметр max_iter: 100 Параметр C: 0.5
CPU times: user 45.5 s, sys: 1min 51s, total: 2min 36s
Wall time: 2min 37s


In [15]:
probabilities_valid = best_model_regression.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.7881861508270256


Обучим и найдем F1-меру для дерева решений 

In [16]:
%%time

best_model_tree = None
best_f1_tree = 0
best_depth_tree = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    f1_tree = f1_score(target_valid, predictions_valid)
    if f1_tree > best_f1_tree:
        best_model_tree = model
        best_f1_tree = f1_tree
        best_depth_tree = depth

print(f'Лучший параметр F1 {best_f1_tree}, значение глубины дерева {best_depth_tree}')

Лучший параметр F1 0.5942684766214177, значение глубины дерева 9
CPU times: user 146 ms, sys: 0 ns, total: 146 ms
Wall time: 146 ms


In [17]:
probabilities_valid = best_model_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8037790860667227


Обучим и найдем F1-меру для случайного леса

In [18]:
%%time

best_model_forest = None
best_f1_forest = 0
best_depth_forest = 0
best_est = 0
for est in range(1, 41):
    for depth in range(1, 14):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        f1_forest = f1_score(target_valid, predictions_valid)
        if f1_forest > best_f1_forest:
            best_model_forest = model
            best_f1_forest = f1_forest
            best_depth_forest = depth
            best_est = est

print(f'Лучший параметр F1 {best_f1_forest}, \
значение числа деревьев {best_est}, значение глубины дерева {best_depth_forest}')

Лучший параметр F1 0.5664335664335663, значение числа деревьев 38, значение глубины дерева 12
CPU times: user 47 s, sys: 130 ms, total: 47.2 s
Wall time: 47.2 s


Изначально я брал я параметр est равным от 1 до 100 с шагом 1 и параметр depth от 1 до 20 с шагом 1. Чтобы время выполнение ячейки понизилось с 10 минут до 45 секунд я уменьшил параметры до наблюдаемых. Итоговый результат остался таким же.  

In [19]:
probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8547986169516868


Лучший результат для метрики F1 = 0.5942684766214177 показал случайный лес с глубиной 9, а лучший результат для метрики AUC-ROC - 0.8547986169516868 показал случайный лес со значениями числа деревьев 38 и глубиной 12.  

## Борьба с дисбалансом

Для борьбы с дисбалансом необходимо выровнять количество значений равных 0 и 1. Существует много различных способов борьбы с дисбалансом, для исследования мы применим: Увеличение меньшей выборки и уменьшение большей выборки.

Для начала исследуем логистическую регрессию, уровняя вес классов в параметре class_weight

In [20]:
%%time

best_f1_regression = 0

for iter in range(100, 3001, 100):
    model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter, class_weight='balanced') 
    model_regression.fit(features_train, target_train)
    predictions_valid = model_regression.predict(features_valid)
    f1_regression = f1_score(target_valid, predictions_valid)
    if f1_regression > best_f1_regression:
        best_model_regression = model_regression
        best_f1_regression = f1_regression
        best_max_iter = iter
print('Параметр F1:', best_f1_regression, 'Параметр max_iter:', best_max_iter)

Параметр F1: 0.5092322643343051 Параметр max_iter: 100
CPU times: user 3.33 s, sys: 8.99 s, total: 12.3 s
Wall time: 12.3 s


In [21]:
probabilities_valid = model_regression.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.7880796187272218


Значение F1 сразу выросло с 0.32719836400818 до 0.5092322643343051!

Найдем отношение класса со значением 0 к классу со значением 1 

Исследуем дерево решений, уровняя вес классов в параметре class_weight

In [22]:
%%time

best_model_tree = None
best_f1_tree = 0
best_depth_tree = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced') 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    f1_tree = f1_score(target_valid, predictions_valid)
    if f1_tree > best_f1_tree:
        best_model_tree = model
        best_f1_tree = f1_tree
        best_depth_tree = depth

print(f'Лучший параметр F1 {best_f1_tree}, значение глубины дерева {best_depth_tree}')

Лучший параметр F1 0.5741728922091781, значение глубины дерева 6
CPU times: user 172 ms, sys: 0 ns, total: 172 ms
Wall time: 172 ms


In [23]:
probabilities_valid = best_model_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8272096065788244


Значение F1 выросло с 0.5664335664335663 до 0.5741728922091781

Исследуем случайный лес, уровняя вес классов в параметре class_weight

In [24]:
%%time

best_model_forest = None
best_f1_forest = 0
best_depth_forest = 0
best_est = 0
for est in range(1, 101):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced') 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        f1_forest = f1_score(target_valid, predictions_valid)
        if f1_forest > best_f1_forest:
            best_model_forest = model
            best_f1_forest = f1_forest
            best_depth_forest = depth
            best_est = est

print(f'Лучший параметр F1 {best_f1_forest}, \
значение числа деревьев {best_est}, значение глубины дерева {best_depth_forest}')

Лучший параметр F1 0.6191709844559586, значение числа деревьев 96, значение глубины дерева 9
CPU times: user 3min 16s, sys: 894 ms, total: 3min 17s
Wall time: 3min 17s


In [25]:
probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8576992804410803


Значение F1 выросло с 0.5664335664335663 до 0.6191709844559586. Это хороший показатель, возможно стоит принять эту модель для финального тестирования

In [26]:
target[target == 0].count() / target[target == 1].count()

3.9034519956850056

"Нулей" больше "едениц" в 3.9 раз

### Увеличение выборки  

Функция для увеличения выбоки для класса со значением 1

In [27]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Баланс классов:

In [28]:
target_upsampled[target == 0].count() / target_upsampled[target == 1].count()

0.9684986595174263

Теперь можно обучать наши модели   

Обучим и найдем F1-меру для логистической регрессии 

In [29]:
%%time

best_f1_regression = 0

for iter in range(100, 3001, 100):
    model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
    model_regression.fit(features_upsampled, target_upsampled)
    predictions_valid = model_regression.predict(features_valid)
    f1_regression = f1_score(target_valid, predictions_valid)
    if f1_regression > best_f1_regression:
        best_model_regression = model_regression
        best_f1_regression = f1_regression
        best_max_iter = iter
print('Параметр F1:', best_f1_regression, 'Параметр max_iter:', best_max_iter)

Параметр F1: 0.5085714285714286 Параметр max_iter: 100
CPU times: user 2.69 s, sys: 5.39 s, total: 8.08 s
Wall time: 8.05 s


In [30]:
probabilities_valid = model_regression.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.7880908326324643


Обучим и найдем F1-меру для дерева решений 

In [31]:
%%time

best_model_tree = None
best_f1_tree = 0
best_depth_tree = 0
for depth in range(1, 21):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_upsampled, target_upsampled) 
    predictions_valid = model.predict(features_valid)
    f1_tree = f1_score(target_valid, predictions_valid)
    if f1_tree > best_f1_tree:
        best_model_tree = model
        best_f1_tree = f1_tree
        best_depth_tree = depth

print(f'Лучший параметр F1 {best_f1_tree}, значение глубины дерева {best_depth_tree}')

Лучший параметр F1 0.5741728922091781, значение глубины дерева 6
CPU times: user 563 ms, sys: 0 ns, total: 563 ms
Wall time: 568 ms


In [32]:
probabilities_valid = best_model_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8272096065788244


Обучим и найдем F1-меру для случайного леса

In [33]:
%%time

best_model_forest = None
best_f1_forest = 0
best_depth_forest = 0
best_est = 0
for est in range(1, 15):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_upsampled, target_upsampled) 
        predictions_valid = model.predict(features_valid)
        f1_forest = f1_score(target_valid, predictions_valid)
        if f1_forest > best_f1_forest:
            best_model_forest = model
            best_f1_forest = f1_forest
            best_depth_forest = depth
            best_est = est

print(f'Лучший параметр F1 {best_f1_forest}, \
значение числа деревьев {best_est}, значение глубины дерева {best_depth_forest}')

Лучший параметр F1 0.6180159635119727, значение числа деревьев 11, значение глубины дерева 7
CPU times: user 6.48 s, sys: 39.1 ms, total: 6.52 s
Wall time: 6.53 s


In [34]:
probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8562489486963836


### Уменьшение выборки  

Функция для уменьшения выбоки для класса со значением 0

In [35]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.259)

In [36]:
target_downsampled[target == 0].count() / target_downsampled[target == 1].count()

1.003574620196604

Теперь можно обучать наши модели   

Обучим и найдем F1-меру для логистической регрессии 

In [37]:
%%time

best_f1_regression = 0

for iter in range(100, 3001, 100):
    model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
    model_regression.fit(features_downsampled, target_downsampled)
    predictions_valid = model_regression.predict(features_valid)
    f1_regression = f1_score(target_valid, predictions_valid)
    if f1_regression > best_f1_regression:
        best_model_regression = model_regression
        best_f1_regression = f1_regression
        best_max_iter = iter
print('Параметр F1:', best_f1_regression, 'Параметр max_iter:', best_max_iter)

Параметр F1: 0.5197628458498023 Параметр max_iter: 100
CPU times: user 1.5 s, sys: 3.21 s, total: 4.71 s
Wall time: 4.7 s


In [38]:
probabilities_valid = model_regression.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.7875918138491731


Обучим и найдем F1-меру для дерева решений 

In [39]:
%%time

best_model_tree = None
best_f1_tree = 0
best_depth_tree = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_downsampled, target_downsampled) 
    predictions_valid = model.predict(features_valid)
    f1_tree = f1_score(target_valid, predictions_valid)
    if f1_tree > best_f1_tree:
        best_model_tree = model
        best_f1_tree = f1_tree
        best_depth_tree = depth

print(f'Лучший параметр F1 {best_f1_tree}, значение глубины дерева {best_depth_tree}')

Лучший параметр F1 0.5758354755784061, значение глубины дерева 5
CPU times: user 111 ms, sys: 62.4 ms, total: 173 ms
Wall time: 177 ms


In [40]:
probabilities_valid = best_model_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8173114662181105


Обучим и найдем F1-меру для случайного леса

In [41]:
%%time

best_model_forest = None
best_f1_forest = 0
best_depth_forest = 0
best_est = 0
for est in range(1, 21):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_downsampled, target_downsampled) 
        predictions_valid = model.predict(features_valid)
        f1_forest = f1_score(target_valid, predictions_valid)
        if f1_forest > best_f1_forest:
            best_model_forest = model
            best_f1_forest = f1_forest
            best_depth_forest = depth
            best_est = est

print(f'Лучший параметр F1 {best_f1_forest}, \
значение числа деревьев {best_est}, значение глубины дерева {best_depth_forest}')

Лучший параметр F1 0.6148491879350348, значение числа деревьев 14, значение глубины дерева 4
CPU times: user 6.27 s, sys: 52.9 ms, total: 6.33 s
Wall time: 6.34 s


In [42]:
probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print(auc_roc)

0.8552340902719373


Лучше всего себя показала модель случайного леса при увеличении выборки с параметрами: число деревьев – 14, глубина дерева – 4. Ее мы и возьмем для финального тестирования

## Тестирование модели

Объединим обучающую и валидационную выборку 

Проверим дисбаланс валидационной выборки

In [43]:
target_valid[target == 0].count() / target_valid[target == 1].count()

3.9295392953929538

Увеличим меньшую выборку

In [44]:
features_upsampled_valid, target_upsampled_valid = upsample(features_valid, target_valid, 4)
target_upsampled_valid[target == 0].count() / target_upsampled_valid[target == 1].count()

0.9823848238482384

In [45]:
features_upsampled_valid_concat = pd.concat([features_upsampled] + [features_upsampled_valid])
target_upsampled_valid_concat = pd.concat([target_upsampled] + [target_upsampled_valid])

Обучим модель случайного леса  

In [46]:
model = RandomForestClassifier(random_state=12345, n_estimators=14, max_depth=4)
model.fit(features_upsampled_valid_concat, target_upsampled_valid_concat) 
predictions_test = model.predict(features_test)
f1_forest = f1_score(target_test, predictions_test)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
f1_forest, auc_roc

(0.574585635359116, 0.8346881633021722)

Лучшая модель на приобраззованных выборка не смогла достигнуть нужного порога, поэтому возьмем модель которая обучалась на выборках с дисбалансом

In [47]:
features_valid_concat = pd.concat([features_train] + [features_valid])
target_valid_concat = pd.concat([target_train] + [target_valid])

In [48]:
model = RandomForestClassifier(random_state=12345, n_estimators=96, max_depth=9, class_weight='balanced') 
model.fit(features_valid_concat, target_valid_concat) 
predictions_test = model.predict(features_test)
f1_forest = f1_score(target_test, predictions_test)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
f1_forest, auc_roc

(0.6140127388535032, 0.8559213596471421)

**Вывод:**

На тестовой выборке модель случайного леса показала F1-меру равную 0.6140127388535032, это значение больше необходимого значения - 0.59, можно сказать что полученная модель прошла тест.

На тестовой выборке модель случайного леса показала значение AUC-ROC равной 0.8559213596471421. Значение AUC-ROC случайной модели равно 0.5. Это означает что полученная модель предсказывает лучше случайной модели на 0.3559213596471421 


# Итоговой вывод

В первом разделе “Подготовка данных” был загружен датасет, на котором основывается все наше исследование. Во время изучения было обнаружено, что датасет имеет около 9% данных с пропусками. Заполнять пропуски показалось плохой идеей, так как искусственные данные могли бы повлиять на результаты обучения модели. Датасет был разделен на две выборки обучения, а впоследствии еще раз разделен на обучающую, валидационную и тестовую выборку. Категориальные признаки были преобразованы, количественные признаки были стандартизированы.

Во втором разделе “Исследование задачи”для обучения были взяты 3 модели: Логистическая регрессия, дерево решений и случайный лес. Лучше всех себя показало дерево решений с параметрами значения глубины дерева – 9, его значение F1-меры оказалось выше всех и составляло 0,5942684766214177. В датасете наблюдался явный дисбаланс классов, поэтому чтобы поднять значение F1-меры необходимо было решить эту проблему. 

В третьем разделе “Борьба с дисбалансом” протестированы 3 различных вариации принятых моделей: 
* Без преобразования выборок и с дополнительным параметром class_weight.
* Разделение выборок на классы и увеличение выборки с классом меньшего веса. 
* Разделение выборок на классы и уменьшение выборки с классом большего веса.

Лучше всего себя показала модель случайного леса без преобразования выборок и с дополнительным параметром class_weight и параметрами значения числа деревьев – 96 и значения глубины дерева - 9, его значение F1-меры составило 0,6191709844559586. Эта модель была выбрана для финального тестирования. 

 В четвертом разделе “Тестирование модели” наша выбранная модель случайного леса дополнительно обучилась на валидационной выборки и показала итоговую F1- меру – 0,6140127388535032. Порог для удачного прохождения теста F1-меры – 0.59. Модель смогла превысить порог на 0.024. Тестирование пройдено. 
